In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import LSTM, Dense, Dropout

/Users/ConCa/Documents/DeepLearning/Assignment/LSTM/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('climate.csv')

data = df.iloc[:, [1,2,5,7,8,10,11]].values


print(data[5])
print(len(data))

[ 9.96500e+02 -8.05000e+00  9.44000e+01  3.14000e+00  1.90000e-01
  3.15000e+00  1.30786e+03]
420551


In [3]:
def DataNormalize(data, train_split):
    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)
    return (data-data_mean)/data_std

In [4]:
train_split = int(0.715*int(df.shape[0]))
print(train_split)

data = DataNormalize(data, train_split)
# print(len(data))

train_data = data[:train_split]
val_data = data[train_split:]


print(len(train_data))
print(len(val_data))

300693
300693
119858


In [5]:
past = 720
future = 72
step = 6
batch_size = 256

start = past + future # start = 792
end = start + train_split # end = 792 + 300693 

X_train = train_data # Dữ liệu từ quá khứ
y_train = data[start:end] # Dữ liệu nhãn bắt đầu từ 792 -> 792+ 300963

sequence_length = int(past/step)
print(sequence_length)

X_end = len(val_data) - past - future

label_start = train_split + past + future

X_val = val_data[:X_end]
y_val = data[label_start:]

120


In [6]:
dataset_train = tf.keras.preprocessing.timeseries_dataset_from_array(X_train, y_train, 
                                                                     sequence_length=sequence_length, 
                                                                     sampling_rate=step, 
                                                                     batch_size=batch_size)
dataset_val = tf.keras.preprocessing.timeseries_dataset_from_array(X_val, y_val, 
                                                                   sequence_length=sequence_length, 
                                                                   sampling_rate=step, 
                                                                   batch_size=batch_size)

2024-10-27 12:37:27.081242: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-10-27 12:37:27.081267: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-10-27 12:37:27.081276: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-10-27 12:37:27.081296: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-27 12:37:27.081310: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
dataset_train

<_BatchDataset element_spec=(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, 7), dtype=tf.float64, name=None))>

In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(sequence_length, 7)))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(1))



model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16)             │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,553 (6.07 KB)

 Trainable params: 1,553 (6.07 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer='Adam', loss='mse')
epochs = 10
history = model.fit(dataset_train, epochs=epochs, validation_data=dataset_val)

Epoch 1/10


2024-10-27 12:37:32.019524: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1172/1172 ━━━━━━━━━━━━━━━━━━━━ 31s 26ms/step - loss: 0.9320 - val_loss: 0.8474
Epoch 2/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - loss: 0.9083 - val_loss: 0.8458
Epoch 3/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - loss: 0.9076 - val_loss: 0.8451
Epoch 4/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 30s 26ms/step - loss: 0.9071 - val_loss: 0.8446
Epoch 5/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 31s 26ms/step - loss: 0.9068 - val_loss: 0.8440
Epoch 6/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 33s 28ms/step - loss: 0.9066 - val_loss: 0.8437
Epoch 7/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 973s 830ms/step - loss: 0.9064 - val_loss: 0.8436
Epoch 8/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 3588s 3s/step - loss: 0.9063 - val_loss: 0.8435
Epoch 9/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 29s 25ms/step - loss: 0.9062 - val_loss: 0.8434
Epoch 10/10
1172/1172 ━━━━━━━━━━━━━━━━━━━━ 556s 475ms/step - loss: 0.9061 - val_loss: 0.8432


In [ ]:
plt.figure(figsize=(12, 8))

# Plot the loss graph
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()